In [ ]:
import pandas as pd
import numpy as np

In [ ]:
label_cols = ["Adequate", "Effective", "Ineffective"]

In [ ]:
folds = [
  # 0,
  #   1,
  #    2,
  #    3,
  #     4,
    "ff"
]

for FOLD in folds:
    df = pd.read_csv("../data/train_folded.csv")
    df_old = pd.read_csv("../data/old_competition_data.csv")
    df_old["discourse_type_essay"] = df_old.groupby("essay_id")["discourse_type"].transform(lambda x: " ".join(x)).values
    ps = []
    for seed in range(1,2):
        if FOLD == "ff":
            df_pseudo = pd.read_csv(f"../data/pseudo_140_ff_raw.csv")
        else:
            df_pseudo = pd.read_csv(f"../data/pseudo_140_fold_{FOLD}_raw.csv")
        ps.append(df_pseudo[label_cols])

    p = np.mean(ps, axis=0)

    df_pseudo[label_cols] = p

    df_pseudo["fold"] = -1
    
    print(p.mean(axis=0))
    
    df_pseudo = df_pseudo.rename(columns={k: "discourse_effectiveness_"+k for k in label_cols})
    
    y = np.zeros((len(df), 3))

    for ii, jj in enumerate([label_cols.index(x) for x in df["discourse_effectiveness"].values]):
        y[ii,jj] = 1
        
    for i, l in enumerate(label_cols):
        df["discourse_effectiveness_"+l] = y[:,i]
        
    df_pseudo = df_old.merge(df_pseudo, left_on="discourse_id", right_on="discourse_id", how="left")
    
    df_pseudo = pd.concat([df_pseudo, df], axis=0)
    
    # df_pseudo.to_csv(f"../data/pseudo_140_fold_{FOLD}.csv", index=False)